In [1]:
import os
import sys

spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()

import pyspark

In [2]:
number_cores = 8
memory_gb = 8
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [3]:
logs = sc.textFile("./data/nsf/*").cache()

In [4]:
#echo %cd%

In [5]:
#logs.take(10)

In [6]:
import glob

data_path = "./data/nsf"
all_files = glob.glob("./data/nsf/*.xml")

file_names = sc.parallelize(all_files)
#file_names.collect()

In [7]:
#file_names.take(5)

In [8]:
s = './data/nsf\\1200004.xml'
s = s.split("\\")[1]

In [9]:
#s

In [10]:
raw_data = file_names.map(lambda f: (f.split("\\")[1], open(f,mode='r', encoding='utf-8').read()))
#raw_data.take(5)

In [11]:
s = raw_data.take(1)

In [12]:
import xml.etree.ElementTree as ET

In [13]:
elem = ET.fromstring(s[0][1])

In [14]:
#for c1 in elem:
#    for c2 in c1:
#        print(c2.tag)
#        for c3 in c2:
#            print("\t", c3.tag)

In [15]:
import xml.etree.ElementTree as ET
def extractUsefulData(y):
    out = []
    try:
        elem = ET.fromstring(y[1])
        for c1 in elem:
            for c2 in c1:
                if (c2.tag == "AwardTitle"):
                    out.append((c2.tag, c2.text))
                if (c2.tag == "AwardEffectiveDate"):
                    out.append((c2.tag, c2.text))
                if (c2.tag == "AwardAmount"):
                    out.append((c2.tag, c2.text))
                for c3 in c2:
                    if (c2.tag == "Investigator" and c3.tag == "NSF_ID"):
                        out.append((c2.tag, c3.tag, c3.text))
                    if (c2.tag == "Performance_Institution" and c3.tag == "Name"):
                        out.append((c2.tag, c3.tag, c3.text))
                    if (c2.tag == "ProgramElement" and c3.tag == "Text"):
                        out.append((c2.tag, c3.tag, c3.text))
                    if (c2.tag == "Appropriation" and c3.tag == "Name"):
                        out.append((c2.tag, c3.tag, c3.text))
    except:
        out.append((""))
    return out

In [16]:
useful_data = raw_data.map(lambda x: extractUsefulData(x))

In [17]:
e = useful_data.take(1)

In [18]:
import xml.etree.ElementTree as ET
def getUniversity(y):
    try:
        elem = ET.fromstring(y[1])
        for c1 in elem:
            for c2 in c1:
                for c3 in c2:
                    if (c2.tag == "Performance_Institution" and c3.tag == "Name"):
                        return (c3.text, 1)
    except:
        return ("", 1)
    return ("", 1)

In [19]:
universities = raw_data.map(lambda x: getUniversity(x))

In [82]:
#universities.take(10)

In [24]:
#counts = universities.countByKey()

In [81]:
#counts

In [30]:
def sumValues(x, y):
    return (x + y)

In [33]:
uniCounts = universities.reduceByKey(sumValues).sortBy(lambda x: (x[1]), ascending=False)

In [34]:
uniCounts.collect()

[('', 2371),
 ('University of Washington', 1508),
 ('University of Michigan Ann Arbor', 1485),
 ('University of Illinois at Urbana-Champaign', 1455),
 ('Georgia Institute of Technology', 1325),
 ('Purdue University', 1232),
 ('University of Wisconsin-Madison', 1218),
 ('Massachusetts Institute of Technology', 1213),
 ('University of Texas at Austin', 1211),
 ('Pennsylvania State Univ University Park', 1198),
 ('University of Colorado at Boulder', 1164),
 ('Arizona State University', 1161),
 ('Cornell University', 1144),
 ('University of California-Berkeley', 1102),
 ('Stanford University', 1040),
 ('University of Florida', 1019),
 ('University of California-San Diego', 989),
 ('North Carolina State University', 981),
 ('Virginia Polytechnic Institute and State University', 939),
 ('Michigan State University', 939),
 ('University of Minnesota-Twin Cities', 935),
 ('Ohio State University', 911),
 ('Northwestern University', 902),
 ('Columbia University', 874),
 ('University of Arizona', 

### Who has won multiple awards

In [70]:
import xml.etree.ElementTree as ET

def getIDCount(e):
    PIs = []
    try:
        elem = ET.fromstring(e[1])
        for n in elem.findall("./Award/Investigator/NSF_ID"):
            if (n.text != ''):
                PIs.append((n.text,1))
    except:
        PIs.append(('Exception',1))
    return PIs
#getIDCount(s[0])

In [71]:
tmp = raw_data.flatMap(getIDCount)
#tmp.count()

In [72]:
#tmp.take(5)

In [73]:
#getAllCount = tmp.countByKey()
#getAllCount

In [74]:
tmp = raw_data.flatMap(getIDCount)
#tmp.count()

In [77]:
NSF = tmp.reduceByKey(lambda x, y: x + y).sortBy(lambda x: (x[1]), ascending=False).cache()
#NSF.count()

In [80]:
NSF.collect()

[('000206836', 71),
 ('000602561', 69),
 ('000648609', 37),
 ('000430507', 36),
 ('000345373', 35),
 ('000463246', 34),
 ('000489704', 30),
 ('000338553', 27),
 ('000073938', 26),
 ('000192562', 25),
 ('000480870', 25),
 ('000676111', 25),
 ('000727410', 25),
 ('000150499', 24),
 ('000599519', 24),
 ('000333030', 24),
 ('000629876', 23),
 ('000204197', 22),
 ('000162566', 22),
 ('000446874', 22),
 ('000487085', 22),
 ('000614644', 22),
 ('000263397', 22),
 ('000163936', 21),
 ('000234022', 21),
 ('000491181', 21),
 ('000253072', 21),
 ('000575362', 21),
 ('000697291', 21),
 ('000358712', 21),
 ('000284152', 21),
 ('000163942', 21),
 ('000268773', 21),
 ('000263174', 21),
 ('000282492', 20),
 ('000508466', 20),
 ('000350397', 20),
 ('000682043', 20),
 ('000487057', 20),
 ('000099536', 20),
 ('000499952', 20),
 ('000230765', 20),
 ('000096020', 20),
 ('000511836', 20),
 ('000572441', 20),
 ('000427715', 20),
 ('000110341', 19),
 ('000386465', 19),
 ('000381415', 19),
 ('000119337', 19),


In [83]:
useful_data.take(1)

[[('AwardTitle',
   'Collaborative Research: Investigation of Odor-triggered Neuronal Dynamics and Experience-induced Olfactory Learning'),
  ('AwardEffectiveDate', '09/15/2012'),
  ('AwardAmount', '355586'),
  ('Investigator', 'NSF_ID', '000601625'),
  ('Performance_Institution', 'Name', 'University of Arizona'),
  ('ProgramElement', 'Text', 'MATHEMATICAL BIOLOGY'),
  ('ProgramElement', 'Text', 'NIGMS'),
  ('Appropriation', 'Name', 'NSF RESEARCH & RELATED ACTIVIT'),
  ('Appropriation', 'Name', 'NSF RESEARCH & RELATED ACTIVIT'),
  ('Appropriation', 'Name', 'NSF RESEARCH & RELATED ACTIVIT')]]

In [180]:
import xml.etree.ElementTree as ET
def getCat(y):
    out = []
    try:
        elem = ET.fromstring(y[1])
        for c1 in elem:
            for c2 in c1:
                if (c2.tag == "AwardTitle"):
                    out.append(c2.text)
                if (c2.tag == "AwardEffectiveDate"):
                    out.append(int(c2.text.split("/")[2]))
                if (c2.tag == "AwardAmount"):
                    out.append(int(c2.text))
                for c3 in c2:
                    if (c2.tag == "Performance_Institution" and c3.tag == "Name"):
                        return (out.append(c3.text))
    except:
        return ["",0,"",""]
    return out

In [181]:
cats = raw_data.map(lambda x: getCat(x)).cache()

In [182]:
#cats.take(10)

In [183]:
#cats.collect()

In [184]:
cats1 = cats.map(lambda x: (x[0], x[1], x[2], x[3]))#.sortBy(lambda x: (x[1]), ascending=False)

In [185]:
cats1.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 106.0 failed 1 times, most recent failure: Lost task 0.0 in stage 106.0 (TID 574, host.docker.internal, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark-3.0.1-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 605, in main
  File "C:\spark-3.0.1-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 597, in process
  File "C:\spark-3.0.1-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark-3.0.1-bin-hadoop3.2\python\pyspark\rdd.py", line 1440, in takeUpToNumLeft
    yield next(iterator)
  File "C:\spark-3.0.1-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\util.py", line 107, in wrapper
    return f(*args, **kwargs)
  File "<ipytho

In [ ]:
#cats1.collect()